

```markdown
# Convolutional Neural Networks (CNNs): From Theory to Implementation

This Jupyter Notebook is based on the book **[Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/chap6.html)** by **Michael Nielsen**. It provides a hands-on guide to understanding and implementing **Convolutional Neural Networks (CNNs)** using **Theano**. The notebook covers key concepts such as **convolutional layers**, **pooling layers**, **dropout**, and **training CNNs**.

---

## Table of Contents
1. [Introduction](#introduction)  
2. [Convolutional Layers](#convolutional-layers)  
3. [Pooling Layers](#pooling-layers)  
4. [Dropout](#dropout)  
5. [Training a CNN](#training-a-cnn)  
6. [Conclusion](#conclusion)  

---

## 1. Introduction <a id="introduction"></a>

Convolutional Neural Networks (CNNs) are a powerful tool for solving complex problems in computer vision, such as image classification, object detection, and more. In this notebook, we will implement a CNN from scratch using **Theano**, a Python library for efficient numerical computation. We will focus on the following key concepts:

- **Convolutional Layers**: Extracting features from images using convolutional filters.
- **Pooling Layers**: Reducing the spatial dimensions of the feature maps.
- **Dropout**: Preventing overfitting by randomly dropping neurons during training.
- **Training CNNs**: Using stochastic gradient descent (SGD) to optimize the network.

Let's get started by importing the necessary libraries.

```python
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.nnet import conv
from theano.tensor.signal import downsample
```

---

## 2. Convolutional Layers <a id="convolutional-layers"></a>

Convolutional layers are the building blocks of CNNs. They apply a set of filters to the input image to extract features such as edges, textures, and patterns.

```python
class ConvPoolLayer:
    def __init__(self, filter_shape, image_shape, poolsize=(2, 2), activation_fn=T.nnet.sigmoid):
        """
        Initialize a convolutional + pooling layer.
        
        :param filter_shape: Tuple of (number of filters, num input feature maps, filter height, filter width)
        :param image_shape: Tuple of (mini-batch size, num input feature maps, image height, image width)
        :param poolsize: Tuple of (pooling height, pooling width)
        :param activation_fn: Activation function to use (e.g., sigmoid, ReLU)
        """
        self.filter_shape = filter_shape
        self.image_shape = image_shape
        self.poolsize = poolsize
        self.activation_fn = activation_fn
        
        # Initialize weights and biases
        n_out = (filter_shape[0] * np.prod(filter_shape[2:]) / np.prod(poolsize))
        self.W = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=np.sqrt(1.0 / n_out), size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )
        self.b = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=1.0, size=(filter_shape[0],)),
                dtype=theano.config.floatX
            ),
            borrow=True
        )
        self.params = [self.W, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        """Set the input to the layer and compute the output."""
        self.inpt = inpt.reshape(self.image_shape)
        conv_out = conv.conv2d(
            input=self.inpt, filters=self.W, filter_shape=self.filter_shape, image_shape=self.image_shape
        )
        pooled_out = downsample.max_pool_2d(
            input=conv_out, ds=self.poolsize, ignore_border=True
        )
        self.output = self.activation_fn(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))
        self.output_dropout = self.output  # No dropout in convolutional layers
```

---

## 3. Pooling Layers <a id="pooling-layers"></a>

Pooling layers reduce the spatial dimensions of the feature maps, making the network more computationally efficient and less prone to overfitting.

```python
# Pooling is integrated into the ConvPoolLayer class above.
```

---

## 4. Dropout <a id="dropout"></a>

Dropout is a regularization technique that randomly drops neurons during training to prevent overfitting.

```python
def dropout_layer(layer, p_dropout):
    """Apply dropout to a layer."""
    srng = theano.tensor.shared_randomstreams.RandomStreams(
        np.random.RandomState(0).randint(999999)
    )
    mask = srng.binomial(n=1, p=1 - p_dropout, size=layer.shape)
    return layer * T.cast(mask, theano.config.floatX)
```

---

## 5. Training a CNN <a id="training-a-cnn"></a>

We will now train a CNN on the MNIST dataset. The network will consist of convolutional, pooling, and fully connected layers.

```python
class Network:
    def __init__(self, layers, mini_batch_size):
        """Initialize the network with a list of layers."""
        self.layers = layers
        self.mini_batch_size = mini_batch_size
        self.params = [param for layer in self.layers for param in layer.params]
        self.x = T.matrix("x")
        self.y = T.ivector("y")
        
        # Set up the network
        init_layer = self.layers[0]
        init_layer.set_inpt(self.x, self.x, self.mini_batch_size)
        for j in range(1, len(self.layers)):
            prev_layer, layer = self.layers[j - 1], self.layers[j]
            layer.set_inpt(prev_layer.output, prev_layer.output_dropout, self.mini_batch_size)
        self.output = self.layers[-1].output
        self.output_dropout = self.layers[-1].output_dropout

    def SGD(self, training_data, epochs, mini_batch_size, eta, validation_data, test_data, lmbda=0.0):
        """Train the network using mini-batch stochastic gradient descent."""
        training_x, training_y = training_data
        validation_x, validation_y = validation_data
        test_x, test_y = test_data
        
        # Define the cost function and gradients
        l2_norm_squared = sum([(layer.W**2).sum() for layer in self.layers])
        cost = self.layers[-1].cost(self) + 0.5 * lmbda * l2_norm_squared / len(training_data)
        grads = T.grad(cost, self.params)
        updates = [(param, param - eta * grad) for param, grad in zip(self.params, grads)]
        
        # Define functions for training and evaluation
        train_mb = theano.function(
            [T.lscalar()], cost, updates=updates,
            givens={
                self.x: training_x[T.lscalar() * mini_batch_size: (T.lscalar() + 1) * mini_batch_size],
                self.y: training_y[T.lscalar() * mini_batch_size: (T.lscalar() + 1) * mini_batch_size]
            }
        )
        
        # Train the network
        best_validation_accuracy = 0.0
        for epoch in range(epochs):
            for minibatch_index in range(len(training_data) // mini_batch_size):
                cost_ij = train_mb(minibatch_index)
            # Evaluate on validation data
            validation_accuracy = np.mean([
                self.layers[-1].accuracy(self.y, 
                givens={
                    self.x: validation_x[i * mini_batch_size: (i + 1) * mini_batch_size],
                    self.y: validation_y[i * mini_batch_size: (i + 1) * mini_batch_size]
                }) for i in range(len(validation_data) // mini_batch_size)
            ])
            print(f"Epoch {epoch}: Validation Accuracy {validation_accuracy:.2%}")
```

---

## 6. Conclusion <a id="conclusion"></a>

In this notebook, we implemented a Convolutional Neural Network (CNN) from scratch using **Theano**. We covered key concepts such as convolutional layers, pooling layers, dropout, and training CNNs. This implementation provides a solid foundation for understanding how CNNs work and can be extended to more complex architectures.

For further reading, check out the book **[Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/chap6.html)** by **Michael Nielsen**.

---

**References**:
- Michael Nielsen, *Neural Networks and Deep Learning*, 2015.
- [Theano Documentation](http://deeplearning.net/software/theano/)
```

---

This notebook is designed to be interactive and educational. You can run each code cell to see the results and experiment with different parameters. Let me know if you'd like further enhancements! 😊